# Lab1 

TODO : Intro

## Step1 :  read the text file

In [122]:
import re

def readAFile(file_name):
    text_file = open(file_name, "r", encoding="utf8")
    next(text_file) #Pass the first line
    lines = text_file.read().splitlines() #Contains raw lines
    dataset = []; #List of tuple (note, list of word)
    
    for line in lines :
        line = re.sub('\?|\!|\/|\;|\:|<br|>|\,|\"|[()]', '', line)
        #print(line)
        current_elt = line.split()
        note = 1 if (float(current_elt[-1]) > 3.0) else -1        
        dataset.append((note,current_elt[1:-1]))
    text_file.close()
    return dataset

training_set = readAFile("lab_train.txt")
for i in range(len(training_set)):
    print(training_set[i])
 


(-1, ['I', 'begin', "I'd", 'just', 'like', 'point', 'out', 'that', 'I', 'am', 'not', 'reviewing', 'this', 'film', 'as', 'a', 'work', 'of', 'art', '--', 'on', 'that', 'score', 'it', 'seems', 'just', 'as', 'good', 'as', 'most', 'films', 'if', 'not', 'at', 'least', 'a', 'little', 'better', '--', 'but', 'as', 'a', 'work', 'of', 'propaganda.', 'I', 'will', 'also', 'try', 'to', 'avoid', 'the', 'mistake', 'made', 'by', '99%', 'of', 'the', 'customer', 'reviewers', 'on', 'Amazon', 'who', 'think', 'they', 'have', 'to', 'recount', 'the', 'plot', 'blow-by-blow', 'in', 'excruciating', 'detail', 'and', 'often', 'erroneous', 'detail', 'even', 'though', 'a', 'hundred', 'other', 'reviewers', 'have', 'already', 'done', 'so', 'and', 'even', 'though', 'there', 'is', 'already', 'a', 'much', 'better', 'more', 'consice', 'and', 'more', 'accuarate', 'Editorial', 'Review', 'at', 'the', 'top', 'of', 'every', 'product', 'page....', 'A', 'couple', 'of', 'years', 'before', '911', 'Tom', 'Clancy', 'wrote', 'a', 'no

## Step2 : Generate the priors probabilities

We will then generate a dictionnary that contains the number of occurences of a word $d_i$ in class $C$

In [123]:
def getCountTable(dataset):
    count_table = {}
    for data in dataset:
        current_class = data[0]
        if (current_class not in count_table):
            count_table[current_class]={}
        for word in data[1]:
            if (word in count_table[current_class]):
                count_table[current_class][word]+=1
            else :
                 count_table[current_class][word]=1
    return count_table
count_table = getCountTable(training_set)
print(count_table)
    

{-1: {'I': 154, 'begin': 3, "I'd": 3, 'just': 43, 'like': 36, 'point': 7, 'out': 23, 'that': 117, 'am': 6, 'not': 64, 'reviewing': 1, 'this': 114, 'film': 36, 'as': 51, 'a': 183, 'work': 8, 'of': 189, 'art': 3, '--': 7, 'on': 55, 'score': 1, 'it': 94, 'seems': 5, 'good': 16, 'most': 14, 'films': 8, 'if': 29, 'at': 29, 'least': 6, 'little': 5, 'better': 10, 'but': 61, 'propaganda.': 2, 'will': 19, 'also': 9, 'try': 4, 'to': 229, 'avoid': 1, 'the': 448, 'mistake': 2, 'made': 11, 'by': 18, '99%': 1, 'customer': 1, 'reviewers': 2, 'Amazon': 2, 'who': 34, 'think': 17, 'they': 34, 'have': 51, 'recount': 1, 'plot': 14, 'blow-by-blow': 1, 'in': 106, 'excruciating': 1, 'detail': 2, 'and': 223, 'often': 1, 'erroneous': 1, 'even': 16, 'though': 3, 'hundred': 2, 'other': 11, 'already': 9, 'done': 7, 'so': 46, 'there': 19, 'is': 154, 'much': 13, 'more': 22, 'consice': 1, 'accuarate': 1, 'Editorial': 1, 'Review': 1, 'top': 2, 'every': 3, 'product': 2, 'page....': 1, 'A': 4, 'couple': 3, 'years': 5, 

We then need to calculate the probabilities of a given class $P(C)$ and $N_C$ the total number of words in class $C$

In [124]:
def getClassProbabilities(dataset):
    number_of_data = len(dataset)
    result = {}
    #Count all occurence of each class
    for data in dataset:
        current_class = data[0]
        if (current_class in result):
            result[current_class]+=1
        else :
            result[current_class]=1
    #Divide by the total number to get a probability
    for Class in result:
        result[Class] = result[Class]/number_of_data            
    return result

P_C = getClassProbabilities(training_set)
print(P_C)

{-1: 0.21, 1: 0.79}


In our example we have $ P(1) = 0.5637877504939317 $ and $P(0) = 0.4362122495060683$

In [125]:
def getNumberOfWordInClass(dataset):
    result = {}
    for data in dataset:
        current_class = data[0]
        if (current_class not in result):
            result[current_class]=0
        result[current_class] += len(data[1])
    return result

N_C = getNumberOfWordInClass(training_set)
print(N_C)

{-1: 8713, 1: 25800}


In our example we have $ N_1 = 42082 $ and $N_0 = 35060$

## Step 3 : Try the classifier

The probability of a document belongs to a class $C$ is given by
$$P = P(C) \cdot \prod_i P(d_i|C) = P(C) \cdot \prod_i^n \frac{count(d_i, C)}{\sum_j count(d_j, C)} = P(C) \cdot \prod_i \frac{count(d_i, C)}{N_C} $$
If we don't have any probability for the word $d_i$, we use $P(d_i|C) =1$ That is called the **Laplacian Smoothing**

### 3.1 Get the test dataset

In [126]:

test_dataset = readAFile("lab_test.txt")
print(test_dataset[0])

(1, ['I', 'thank', 'the', 'Lord', 'for', 'this', 'DVD', 'and', 'movie...It', 'has', 'touched', 'my', 'life', 'down', 'through', 'the', 'years..in', 'fact', 'when', 'I', 'was', 'in', 'a', 'car', 'accident', 'I', 'was', 'playing', 'the', 'music', 'from', 'this', 'movie', 'on', 'an', '8', 'track', 'tape', 'and', 'it', 'was', 'a', 'blessing', 'as', 'I', 'was', 'hit', 'and', 'in', 'a', 'ditch..the', 'song', 'was', "playing..It's", 'A', 'Miracle', 'and', 'it', 'was..thank', 'you'])


### 3.2 Choose according to the Naïve Bayes Classifier

In [127]:
def get_max_value_key(dictionnary):
    values = list(dictionnary.values())
    keys = list(dictionnary.keys())
    max_value_index = values.index(max(values))
    max_key = keys[max_value_index]
    return max_key

def classify(list_of_word):
    prediction = {}
    nb_word_counted = 0
    for Classe in N_C :
        prediction[Classe] = P_C[Classe] #Initialisation 
        for word in list_of_word :
            if (word in count_table[Classe]):
                nb_word_counted +=1
                print(prediction) #Problème ici : float trop grand... Solution?
                prediction[Classe]*=float(count_table[Classe][word]/N_C[Classe])
   
    return get_max_value_key(prediction)

index_to_try = 12
print(classify(test_dataset[index_to_try][1]))
print(' '.join(test_dataset[index_to_try][1]))
print("Real class : ",test_dataset[index_to_try][0])
                
        

{-1: 0.21}
{-1: 0.0037116951681395614}
{-1: 0.00019084579769614637}
{-1: 1.971321220320575e-07}
{-1: 4.2761357814827125e-09}
{-1: 9.815530314432944e-13}
{-1: 2.0615655314371958e-14}
{-1: 8.04467210706584e-17}
{-1: 3.6931812726114263e-20}
{-1: 2.331286238880161e-22}
{-1: 4.8964235247913405e-24}
{-1: 6.74361096034616e-27}
{-1: 2.0123250885917613e-29}
{-1: 9.238265068710025e-33}
{-1: 1.0602852138999225e-36}
{-1: 2.433800559852915e-40}
{-1: 2.7932980142923393e-44}
{-1: 2.564717561613533e-47}
{-1: 5.5633148071267965e-49}
{-1: 2.860513065066917e-50}
{-1: 2.659262691041206e-52}
{-1: 4.7001773719768817e-54}
{-1: 2.4167100455017134e-55}
{-1: 2.4963147491696795e-58}
{-1: 5.414937307392051e-60}
{-1: 1.6158426488258157e-62}
{-1: 5.563557840557153e-66}
{-1: 5.108282190342846e-69}
{-1: 6.214597867282701e-71}
{-1: 3.1953860261019736e-72}
{-1: 1.8336887559405336e-75}
{-1: 4.2090870100781215e-79}
{-1: 5.652050730852062e-81}
{-1: 5.838225247063991e-84}
{-1: 1.0318910685732293e-85}
{-1: 1.184312026366612

If we try to display all the test data :

In [104]:
for data in test_dataset:
    choosen_class = classify(data)
    print("----------------------------------")
    print(' '.join(data))
    if (choosen_class=='1') :
        print(choosen_class,"- Positive")
    else :
        print(choosen_class,"- Negative")
    

----------------------------------
The check in staff were very friendly and could not do enough to help after we asked them to change the bedding as we were both allergic to feather duvets etc They had made a note on our last visit, and could not do enough to rectify once we told them they were still feather!
0 - Negative
----------------------------------
The room was great - modern & clean. Robes & slippers were a welcome touch (superior room) The restaurant area had been refurbished and was very attractive. The breakfast was fabulous. The staff were friendly although there could have been more staff on duty on Saturday afternoon. A large hen party group was checking in and there was no porter on duty so to collect bags from the storage area took over 20 minutes.
1 - Positive
----------------------------------
This is a great hotel. The staff are very friendly and professional and willing to help. The rooms have everything you need for a business traveler. The wifi is quick and I ha